# Decipher Single-byte XOR using ETAOIN SHRDLU

https://www.insider.com/gchq-reveals-last-nazi-message-wwii-2020-5

![final-message](https://i.insider.com/5eb52a95fc593d494f70c193?width=500&format=jpeg&auto=webp)

In [1]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
plain_text = b'british troops entered cuxhaven at 1400 on 6 may - from now on all radio traffic will cease - wishing you all the best. lt kunkel.'

In [3]:
key = random.randint(0, 255)

In [4]:
def encrypt_single_xor(text: bytes, key: int) -> bytes:
    return bytes([b ^ key for b in text])

In [5]:
cipher_text = encrypt_single_xor(plain_text, key)

Now we need to find `key` and `plain_text` given this `cipher_text`

In [6]:
def decipher_single_xor(text: bytes, key: int) -> bytes:
    return bytes([b ^ key for b in text])

In [7]:
def decipher(text: bytes, scoring_fn=lambda x: 1):
    plains = []
    for _key in range(256):
        _plain_text = decipher_single_xor(text, _key)
        _score = scoring_fn(_plain_text)
        plains.append((_plain_text, _key, _score))
    return sorted(plains, key=lambda x: x[2], reverse=False)

In [8]:
def plot_linears(x1, y1, x2, y2):
    plt.plot(x1, y1)
    plt.plot(x2, y2)
    plt.show()

In [9]:
def scoring_fn_dummy(text: bytes):
    return 1

In [10]:
plains = decipher(cipher_text, scoring_fn=scoring_fn_dummy)

In [11]:
pd.DataFrame(plains, columns=["Plain Text", "Key", "Score"])

,Plain Text,Key,Score
0,b'\x9e\x8e\x95\x88\x95\x8f\x94\xdc\x88\x8e\x93...,0,1
1,b'\x9f\x8f\x94\x89\x94\x8e\x95\xdd\x89\x8f\x92...,1,1
2,b'\x9c\x8c\x97\x8a\x97\x8d\x96\xde\x8a\x8c\x91...,2,1
3,b'\x9d\x8d\x96\x8b\x96\x8c\x97\xdf\x8b\x8d\x90...,3,1
4,b'\x9a\x8a\x91\x8c\x91\x8b\x90\xd8\x8c\x8a\x97...,4,1
...,...,...,...
251,"b""eunsnto'suhhwt'bisbubc'dr\x7fofqbi'fs'6377'h...",251,1
252,b'british troops entered cuxhaven at 1400 on 6...,252,1
253,b'cshuhri!usnnqr!doudsde!btyi`wdo!`u!0511!no!7...,253,1
254,"b'`pkvkqj""vpmmrq""glvgpgf""awzjctgl""cv""3622""ml""4...",254,1


In [12]:
def scoring_fn_maxfit(text: bytes):
    freqs = {
        'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253,
        'e': 0.12702, 'f': 0.02228, 'g': 0.02015, 'h': 0.06094,
        'i': 0.06094, 'j': 0.00153, 'k': 0.00772, 'l': 0.04025,
        'm': 0.02406, 'n': 0.06749, 'o': 0.07507, 'p': 0.01929,
        'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056,
        'u': 0.02758, 'v': 0.00978, 'w': 0.02360, 'x': 0.00150,
        'y': 0.01974, 'z': 0.00074, # ' ': 0.13000
    }
    counter = Counter(text)
    x1, y1 = list(freqs.keys()), [(counter.get(ord(k), 0) * 100)/len(text) for k in freqs]
    x2, y2 = list(freqs.keys()), [(v * 100)/sum(freqs.values()) for v in freqs.values()]
    return sum([abs(a - b) for a, b in zip(y1, y2)])/len(y2)

In [13]:
plains = decipher(cipher_text, scoring_fn=scoring_fn_maxfit)

In [14]:
plains[0]

(b'british troops entered cuxhaven at 1400 on 6 may - from now on all radio traffic will cease - wishing you all the best. lt kunkel.',
 252,
 1.2800989323181917)

In [15]:
pd.DataFrame(plains, columns=["Plain Text", "Key", "Score"])

,Plain Text,Key,Score
0,b'british troops entered cuxhaven at 1400 on 6...,252,1.280099
1,b'cshuhri!usnnqr!doudsde!btyi`wdo!`u!0511!no!7...,253,2.440608
2,"b""eunsnto'suhhwt'bisbubc'dr\x7fofqbi'fs'6377'h...",251,2.582544
3,b'dtoroun&rtiivu&chrctcb&es~ngpch&gr&7266&ih&0...,250,2.600830
4,b'yirorhs;oittkh;~uo~i~\x7f;xncszm~u;zo;*/++;t...,231,2.618637
...,...,...,...
251,"b')9""?""8#k?9$$;8k.%?.9./k(>3#*=.%k*?kz\x7f{{k$...",183,4.431391
252,"b'4$?""?%>v""$99&%v38""3$32v5#.>7 38v7""vgbffv98v`...",170,4.439094
253,b'{kpmpjq9mkvvij9|wm|k|}9zlaqxo|w9xm9(-))9vw9/...,229,4.449043
254,"b'3#8%8""9q%#>>!""q4?%4#45q2$)90\'4?q0%q`eaaq>?q...",173,4.460084


In [16]:
sentences = [
    b'His mind was blown that there was nothing in space except space itself.',
    b'I love bacon, beer, birds, and baboons.',
    b'With a single flip of the coin, his life changed forever.',
    b'If you like tuna and tomato sauce - try combining the two. It\'s really not as bad as it sounds.',
    b'The view from the lighthouse excited even the most seasoned traveler.',
    b'The small white buoys marked the location of hundreds of crab pots.',
    b'Be careful with that butter knife.',
    b'We have young kids who often walk into our room at night for various reasons including clowns in the closet.',
    b'Mary plays the piano.',
    b'The lake is a long way from here.',
    b'Buried deep in the snow, he hoped his batteries were fresh in his avalanche beacon.',
    b'Swim at your own risk was taken as a challenge for the group of Kansas City college students.',
    b'Peanut butter and jelly caused the elderly lady to think about her past.',
    b'They say that dogs are man\'s best friend, but this cat was setting out to sabotage that theory.',
    b'He found a leprechaun in his walnut shell.',
    b'She traveled because it cost the same as therapy and was a lot more enjoyable.',
    b'Had he known what was going to happen, he would have never stepped into the shower.',
    b'She wanted a pet platypus but ended up getting a duck and a ferret instead.',
    b'The near-death experience brought new ideas to light.',
    b'I was very proud of my nickname throughout high school but today - I couldn\'t be any different to what my nickname was.',
    b'It\'s not often you find a soggy banana on the street.',
    b'The doll spun around in circles in hopes of coming alive.',
    b'You\'re unsure whether or not to trust him, but very thankful that you wore a turtle neck.',
    b'Shakespeare was a famous 17th-century diesel mechanic.',
    b'There\'s a message for you if you look up.',
    b'She had the gift of being able to paint songs.',
    b'Hit me with your pet shark!',
    b'Abstraction is often one floor above you.',
    b'David subscribes to the "stuff your tent into the bag" strategy over nicely folding it.',
    b'The rusty nail stood erect, angled at a 45-degree angle, just waiting for the perfect barefoot to come along.',
    b'Joyce enjoyed eating pancakes with ketchup.',
    b'Let me help you with your baggage.',
    b'The door slammed on the watermelon.',
    b'The tour bus was packed with teenage girls heading toward their next adventure.',
    b'The skeleton had skeletons of his own in the closet.',
    b'25 years later, she still regretted that specific moment.',
    b'There were three sphered rocks congregating in a cubed room.',
    b'The thick foliage and intertwined vines made the hike nearly impossible.',
    b'The stranger officiates the meal.',
    b'She says she has the ability to hear the soundtrack of your life.',
    b'I am counting my calories, yet I really want dessert.',
    b'He had unknowingly taken up sleepwalking as a nighttime hobby.',
    b'I\'m confused: when people ask me what\'s up, and I point, they groan.',
    b'He\'s in a boy band which doesn\'t make much sense for a snake.',
    b'Combines are no longer just for farms.',
    b'There are few things better in life than a slice of pie.',
    b'Greetings from the galaxy MACS0647-JD, or what we call home.',
    b'She was the type of girl who wanted to live in a pink house.',
    b'Smoky the Bear secretly started the fires.',
    b'He didn\'t understand why the bird wanted to ride the bicycle.',
    b'He quietly entered the museum as the super bowl started.',
    b'Nobody loves a pig wearing lipstick.',
    b'Shakespeare was a famous 17th-century diesel mechanic.',
    b'He had decided to accept his fate of accepting his fate.',
    b'She saw no irony asking me to change but wanting me to accept her for who she is.',
    b'The lyrics of the song sounded like fingernails on a chalkboard.',
    b'Various sea birds are elegant, but nothing is as elegant as a gliding pelican.',
    b'The newly planted trees were held up by wooden frames in hopes they could survive the next storm.',
    b'She only paints with bold colors; she does not like pastels.',
    b'The urgent care center was flooded with patients after the news of a new deadly virus was made public.',
    b'The light in his life was actually a fire burning all around him.',
    b'I would have gotten the promotion, but my attendance wasn\'t good enough.',
    b'He had a hidden stash underneath the floorboards in the back room of the house.',
    b'There can never be too many cherries on an ice cream sundae.',
    b'The thunderous roar of the jet overhead confirmed her worst fears.',
    b'We should play with legos at camp.',
    b'He was surprised that his immense laziness was inspirational to others.',
    b'Bill ran from the giraffe toward the dolphin.',
    b'The waves were crashing on the shore; it was a lovely sight.',
    b'Check back tomorrow; I will see if the book has arrived.',
    b'The irony of the situation wasn\'t lost on anyone in the room.',
    b'Her hair was windswept as she rode in the black convertible.',
    b'It was the scarcity that fueled his creativity.',
    b'Everyone says they love nature until they realize how dangerous she can be.',
    b'Sometimes, all you need to do is completely make an ass of yourself and laugh it off to realise that life isn\'t so bad after all.',
    b'You\'re unsure whether or not to trust him, but very thankful that you wore a turtle neck.',
    b'She found his complete dullness interesting.',
    b'The paintbrush was angry at the color the artist chose to use.',
    b'his seven-layer cake only had six layers.',
    b'Even though he thought the world was flat he didn\'t see the irony of wanting to travel around the world.',
    b'She tilted her head back and let whip cream stream into her mouth while taking a bath.',
    b'She hadn\'t had her cup of coffee, and that made things all the worse.',
    b'Flying fish few by the space station.',
    b'The blinking lights of the antenna tower came into focus just as I heard a loud snap.',
    b'He poured rocks in the dungeon of his mind.',
    b'Sometimes it is better to just walk away from things and go back to them later when you\'re in a better frame of mind.',
    b'There\'s an art to getting your way, and spitting olive pits across the table isn\'t it.',
    b'I am counting my calories, yet I really want dessert.',
    b'As you consider all the possible ways to improve yourself and the world, you notice John Travolta seems fairly unhappy.',
    b'After exploring the abandoned building, he started to believe in ghosts.',
    b'He invested some skill points in Charisma and Strength.',
    b'The blue parrot drove by the hitchhiking mongoose.',
    b'Had he known what was going to happen, he would have never stepped into the shower.',
    b'It was a slippery slope and he was willing to slide all the way to the deepest depths.',
    b'Going from child, to childish, to childlike is only a matter of time.',
    b'The door slammed on the watermelon.',
    b'They got there early, and they got really good seats.',
    b'There was coal in his stocking and he was thrilled.',
    b'He is good at eating pickles and telling women about his emotional problems.',
    b'The three-year-old girl ran down the beach as the kite flew behind her.',
]

In [20]:
for sentence in sentences:
    sentence = sentence.lower()
    key = random.randint(1, 255)
    ct = encrypt_single_xor(sentence, key)
    pts = decipher(ct, scoring_fn=scoring_fn_maxfit)
    try:
        assert pts[0][1] == key
    except AssertionError as e:
        print(pts[0][1], key, sentence, pts[0][0], pts.index(list(filter(lambda x: x[1] == key, pts))[0]))